In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Punta Arenas,CL,2020-08-28 18:12:08,-53.15,-70.92,32.00,88,20,36.91
1,1,Vaini,TO,2020-08-28 18:12:08,-21.20,-175.20,73.40,88,40,10.29
2,2,Lagoa,PT,2020-08-28 18:12:09,39.05,-27.98,74.28,90,11,17.78
3,3,Port Elizabeth,ZA,2020-08-28 18:12:09,-33.92,25.57,57.20,67,4,14.99
4,4,Hilo,US,2020-08-28 18:08:41,19.73,-155.09,75.99,73,1,3.36


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [5]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [9]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
1,1,Vaini,TO,2020-08-28 18:12:08,-21.20,-175.20,73.40,88,40,10.29
2,2,Lagoa,PT,2020-08-28 18:12:09,39.05,-27.98,74.28,90,11,17.78
4,4,Hilo,US,2020-08-28 18:08:41,19.73,-155.09,75.99,73,1,3.36
7,7,Atuona,PF,2020-08-28 18:12:10,-9.80,-139.03,77.88,79,48,21.47
8,8,Razdolnoye,RU,2020-08-28 18:12:10,43.55,131.92,71.60,100,40,0.47
9,9,Ribeira Grande,PT,2020-08-28 18:12:10,38.52,-28.70,77.00,88,40,11.41
13,13,Padang,ID,2020-08-28 18:12:11,-0.95,100.35,77.81,79,75,3.87
18,18,Kutum,SD,2020-08-28 18:12:12,14.20,24.67,79.12,55,79,9.51
19,19,Faanui,PF,2020-08-28 18:12:13,-16.48,-151.75,76.51,80,28,22.10
22,22,Kapaa,US,2020-08-28 18:11:11,22.08,-159.32,77.00,88,90,4.70


In [10]:
preferred_cities_df.count()

City_ID       152
City          152
Country       152
Date          152
Lat           152
Lng           152
Max Temp      152
Humidity      152
Cloudiness    152
Wind Speed    152
dtype: int64

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Vaini,TO,73.40,-21.20,-175.20,
2,Lagoa,PT,74.28,39.05,-27.98,
4,Hilo,US,75.99,19.73,-155.09,
7,Atuona,PF,77.88,-9.80,-139.03,
8,Razdolnoye,RU,71.60,43.55,131.92,
9,Ribeira Grande,PT,77.00,38.52,-28.70,
13,Padang,ID,77.81,-0.95,100.35,
18,Kutum,SD,79.12,14.20,24.67,
19,Faanui,PF,76.51,-16.48,-151.75,
22,Kapaa,US,77.00,22.08,-159.32,


In [12]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [16]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [17]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
1,Vaini,TO,73.40,-21.20,-175.20,Keleti Beach Resort
2,Lagoa,PT,74.28,39.05,-27.98,Casa Das Faias
4,Hilo,US,75.99,19.73,-155.09,Hilo Hawaiian Hotel
7,Atuona,PF,77.88,-9.80,-139.03,Villa Enata
8,Razdolnoye,RU,71.60,43.55,131.92,"Baza Otdykha ""Lesnaya"""
9,Ribeira Grande,PT,77.00,38.52,-28.70,Quinta da Meia Eira
13,Padang,ID,77.81,-0.95,100.35,Mercure Padang
18,Kutum,SD,79.12,14.20,24.67,Dibbo areaمنطقة دبو
19,Faanui,PF,76.51,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
22,Kapaa,US,77.00,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach


In [19]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [22]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [23]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))